## 0- Importing the necessary libraries:

In [60]:
import requests
from bs4 import BeautifulSoup
import pymongo
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from nltk.stem import WordNetLemmatizer
from farasa.pos import FarasaPOSTagger  # pip install farasapy
from farasa.ner import FarasaNamedEntityRecognizer 

import re


## 1-  Use libraries like Scrapy and Beautiful Soup to do scraping from server Arabic web sources concerning a specific domaine.

In [16]:
# Variables where we will put our texts (in this case we will create 3 documents in our mongodb database):
arabiccontent1 = ""
arabiccontent2 = ""
arabiccontent3 = ""
title1=""
title2=""
title3=""
# Links of the sites where we got we scrapped from :
link1 = 'https://www.bbc.com/arabic/articles/cd191zxrx5zo'
link2 = 'https://www.bbc.com/arabic/articles/cg3q3q3zjd0o'
link3 = 'https://www.bbc.com/arabic/articles/c876zz8rpp8o'
# Now we are gonna extract the arabic texts from each article with the title as well :
page1 = requests.get(link1)
page2 = requests.get(link2)
page3 = requests.get(link3)
soup1 = BeautifulSoup(page1.text, 'html.parser')
soup2 = BeautifulSoup(page2.text, 'html.parser')
soup3 = BeautifulSoup(page3.text, 'html.parser')
for arabictext in soup1.find_all('p'):
        arabiccontent1 += arabictext.text + "\n"
for arabictext in soup2.find_all('p'):
        arabiccontent2 += arabictext.text + "\n"
for arabictext in soup3.find_all('p'):
        arabiccontent3 += arabictext.text + "\n"
title1 = soup1.find('h1').text
title2 = soup2.find('h1').text
title3 = soup3.find('h1').text

In [23]:
print(title1)
print(arabiccontent1)

ماذا نعرف عن الفلسطيني الذي قُتل ضمن فريق المطبخ المركزي العالمي في غزة؟
سيف قبل ساعات من مقتله
كان سيف على موعد للخروج من غزة في الأسبوع التالي لوفاته، متجها إلى دولة الإمارات ليتقدم لخطبة فتاة، لكنه قُتل خلال قصف إسرائيلي لقافلة إغاثية في قطاع غزة.
وسيف هو الفلسطيني الوحيد الذي قُتل ضمن سبعة موظفين للمطبخ العالمي المركزي في قطاع غزة، بينما كان الآخرون يحملون جنسيات أجنبية متعددة.
يقول عصام أبو طه، والد سيف: "ظهرت أسماؤنا في التنسيق الأمني للخروج من غزة، كنا سنذهب للإمارات أنا وسيف ووالدته لنخطب له، كان فرحا بهذه الخطوة، لكن لا أحد يمكنه منع القدر".
الهجوم الذي وقع الأسبوع الماضي أثار حالة واسعة من الغضب في الأوساط الدولية، واعتذر عنه الجيش الإسرائيلي.
"إذا نسي العالم سيف سيظل موجودا في قلوبنا"، يقول شقيقه عبد الرازق أبو طه في مكالمة هاتفية مع بي بي سي، حيث يحكي عن أخيه بكثير من الفخر، وفي ذات الوقت الأسى لفقدانه.
قصص مقترحة نهاية
كان سيف على موعد للخروج من غزة في الأسبوع التالي لوفاته
اسمه سيف أبو طه يبلغ من العمر 25 عاماً، ولد في قطاع غزة ثم انتقل للدراسة والعمل في الإمارات، تخرج في

## 2- Store the raw data on a NoSql database “MongoDB”.

In [2]:
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017")

In [3]:
db = client["atelier1"]

In [24]:
collection = db["arabicdata"]
Documents = [ {"title" : title1,"content" : arabiccontent1,"url" : link1},
             {"title" : title2,"content" : arabiccontent1,"url" : link2},
             {"title" : title3,"content" : arabiccontent3,"url" : link3}]
results = collection.insert_many(Documents)
print(results)

InsertManyResult([ObjectId('6612fa3bd300a46b0928c66a'), ObjectId('6612fa3bd300a46b0928c66b'), ObjectId('6612fa3bd300a46b0928c66c')], acknowledged=True)


In [27]:
#  Let's verify if our Data has been succesfully inserted
for document in collection.find():
    print(document)

{'_id': ObjectId('6612fa3bd300a46b0928c66a'), 'title': 'ماذا نعرف عن الفلسطيني الذي قُتل ضمن فريق المطبخ المركزي العالمي في غزة؟', 'content': 'سيف قبل ساعات من مقتله\nكان سيف على موعد للخروج من غزة في الأسبوع التالي لوفاته، متجها إلى دولة الإمارات ليتقدم لخطبة فتاة، لكنه قُتل خلال قصف إسرائيلي لقافلة إغاثية في قطاع غزة.\nوسيف هو الفلسطيني الوحيد الذي قُتل ضمن سبعة موظفين للمطبخ العالمي المركزي في قطاع غزة، بينما كان الآخرون يحملون جنسيات أجنبية متعددة.\nيقول عصام أبو طه، والد سيف: "ظهرت أسماؤنا في التنسيق الأمني للخروج من غزة، كنا سنذهب للإمارات أنا وسيف ووالدته لنخطب له، كان فرحا بهذه الخطوة، لكن لا أحد يمكنه منع القدر".\nالهجوم الذي وقع\u202fالأسبوع الماضي أثار حالة واسعة من الغضب في الأوساط الدولية، واعتذر عنه الجيش الإسرائيلي.\n"إذا نسي العالم سيف سيظل موجودا في قلوبنا"، يقول شقيقه عبد الرازق أبو طه في مكالمة هاتفية مع بي بي سي، حيث يحكي عن أخيه بكثير من الفخر، وفي ذات الوقت الأسى لفقدانه.\nقصص مقترحة نهاية\nكان سيف على موعد للخروج من غزة في الأسبوع التالي لوفاته\nاسمه سيف أبو طه ي

## 3- Establishment of NLP Pipeline (Text Cleaning, Tokenization, Stop words, Discretization, Normalization).

In [32]:
# Text Cleaning
def clean_text(text):
    # Removing HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Removing punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    # Removing English letters and numbers
    text = re.sub(r'[a-zA-Z0-9]', '', text)
     # Removing new lines and carriage returns, replace them with a space
    text = re.sub(r'[\n\r]+', ' ', text)
    # multiple spaces into One
    text = re.sub(r'\s+', ' ', text)
    return text
arabiccontent1cleaned = clean_text(arabiccontent1)
arabiccontent2cleaned = clean_text(arabiccontent2)
arabiccontent3cleaned = clean_text(arabiccontent3)
print(arabiccontent1cleaned)

سيف قبل ساعات من مقتله كان سيف على موعد للخروج من غزة في الأسبوع التالي لوفاته متجها إلى دولة الإمارات ليتقدم لخطبة فتاة لكنه قتل خلال قصف إسرائيلي لقافلة إغاثية في قطاع غزة وسيف هو الفلسطيني الوحيد الذي قتل ضمن سبعة موظفين للمطبخ العالمي المركزي في قطاع غزة بينما كان الآخرون يحملون جنسيات أجنبية متعددة يقول عصام أبو طه والد سيف ظهرت أسماؤنا في التنسيق الأمني للخروج من غزة كنا سنذهب للإمارات أنا وسيف ووالدته لنخطب له كان فرحا بهذه الخطوة لكن لا أحد يمكنه منع القدر الهجوم الذي وقع الأسبوع الماضي أثار حالة واسعة من الغضب في الأوساط الدولية واعتذر عنه الجيش الإسرائيلي إذا نسي العالم سيف سيظل موجودا في قلوبنا يقول شقيقه عبد الرازق أبو طه في مكالمة هاتفية مع بي بي سي حيث يحكي عن أخيه بكثير من الفخر وفي ذات الوقت الأسى لفقدانه قصص مقترحة نهاية كان سيف على موعد للخروج من غزة في الأسبوع التالي لوفاته اسمه سيف أبو طه يبلغ من العمر عاما ولد في قطاع غزة ثم انتقل للدراسة والعمل في الإمارات تخرج في جامعة عجمان وعاش هناك لفترة عاد سيف من الإمارات إلى القطاع قبل نحو ثلاثة أعوام حين افتتحت العائلة مشر

In [34]:
# Tokenization
nltk.download('punkt') 
words1 = word_tokenize(arabiccontent1cleaned)
words2 = word_tokenize(arabiccontent2cleaned)
words3 = word_tokenize(arabiccontent3cleaned)
print(words1)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


['سيف', 'قبل', 'ساعات', 'من', 'مقتله', 'كان', 'سيف', 'على', 'موعد', 'للخروج', 'من', 'غزة', 'في', 'الأسبوع', 'التالي', 'لوفاته', 'متجها', 'إلى', 'دولة', 'الإمارات', 'ليتقدم', 'لخطبة', 'فتاة', 'لكنه', 'قتل', 'خلال', 'قصف', 'إسرائيلي', 'لقافلة', 'إغاثية', 'في', 'قطاع', 'غزة', 'وسيف', 'هو', 'الفلسطيني', 'الوحيد', 'الذي', 'قتل', 'ضمن', 'سبعة', 'موظفين', 'للمطبخ', 'العالمي', 'المركزي', 'في', 'قطاع', 'غزة', 'بينما', 'كان', 'الآخرون', 'يحملون', 'جنسيات', 'أجنبية', 'متعددة', 'يقول', 'عصام', 'أبو', 'طه', 'والد', 'سيف', 'ظهرت', 'أسماؤنا', 'في', 'التنسيق', 'الأمني', 'للخروج', 'من', 'غزة', 'كنا', 'سنذهب', 'للإمارات', 'أنا', 'وسيف', 'ووالدته', 'لنخطب', 'له', 'كان', 'فرحا', 'بهذه', 'الخطوة', 'لكن', 'لا', 'أحد', 'يمكنه', 'منع', 'القدر', 'الهجوم', 'الذي', 'وقع', 'الأسبوع', 'الماضي', 'أثار', 'حالة', 'واسعة', 'من', 'الغضب', 'في', 'الأوساط', 'الدولية', 'واعتذر', 'عنه', 'الجيش', 'الإسرائيلي', 'إذا', 'نسي', 'العالم', 'سيف', 'سيظل', 'موجودا', 'في', 'قلوبنا', 'يقول', 'شقيقه', 'عبد', 'الرازق', 'أبو', 'طه', 'في

In [38]:
# Stop Words
nltk.download('stopwords')
stop_words = set(stopwords.words('arabic'))
filtered_words1 = [w for w in words1 if not w.lower() in stop_words]
filtered_words2 = [w for w in words2 if not w.lower() in stop_words]
filtered_words3 = [w for w in words3 if not w.lower() in stop_words]
print(filtered_words1)

['سيف', 'ساعات', 'مقتله', 'سيف', 'موعد', 'للخروج', 'غزة', 'الأسبوع', 'التالي', 'لوفاته', 'متجها', 'دولة', 'الإمارات', 'ليتقدم', 'لخطبة', 'فتاة', 'لكنه', 'قتل', 'خلال', 'قصف', 'إسرائيلي', 'لقافلة', 'إغاثية', 'قطاع', 'غزة', 'وسيف', 'الفلسطيني', 'الوحيد', 'قتل', 'ضمن', 'موظفين', 'للمطبخ', 'العالمي', 'المركزي', 'قطاع', 'غزة', 'بينما', 'الآخرون', 'يحملون', 'جنسيات', 'أجنبية', 'متعددة', 'يقول', 'عصام', 'طه', 'والد', 'سيف', 'ظهرت', 'أسماؤنا', 'التنسيق', 'الأمني', 'للخروج', 'غزة', 'كنا', 'سنذهب', 'للإمارات', 'وسيف', 'ووالدته', 'لنخطب', 'فرحا', 'بهذه', 'الخطوة', 'يمكنه', 'منع', 'القدر', 'الهجوم', 'وقع', 'الأسبوع', 'الماضي', 'أثار', 'حالة', 'واسعة', 'الغضب', 'الأوساط', 'الدولية', 'واعتذر', 'عنه', 'الجيش', 'الإسرائيلي', 'نسي', 'العالم', 'سيف', 'سيظل', 'موجودا', 'قلوبنا', 'يقول', 'شقيقه', 'عبد', 'الرازق', 'طه', 'مكالمة', 'هاتفية', 'سي', 'يحكي', 'أخيه', 'بكثير', 'الفخر', 'وفي', 'الوقت', 'الأسى', 'لفقدانه', 'قصص', 'مقترحة', 'نهاية', 'سيف', 'موعد', 'للخروج', 'غزة', 'الأسبوع', 'التالي', 'لوفاته', 'اسم

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 4- Do the Stemming and Lemmatization, then compare the tow mechanisms.

In [45]:
nltk.download('wordnet')
# STEMMING
stemmer = ISRIStemmer() # specifically designed for the Arabic language.
stemmed_words1 = [stemmer.stem(word) for word in filtered_words1]
stemmed_words2 = [stemmer.stem(word) for word in filtered_words2]
stemmed_words3 = [stemmer.stem(word) for word in filtered_words3]
# LEMMATIZATION
lemmatizer = WordNetLemmatizer()
lemmatized_words1 = [lemmatizer.lemmatize(word) for word in filtered_words1]
lemmatized_words2 = [lemmatizer.lemmatize(word) for word in filtered_words2]
lemmatized_words3 = [lemmatizer.lemmatize(word) for word in filtered_words3]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
print(stemmed_words1)

['سيف', 'ساع', 'قله', 'سيف', 'وعد', 'خرج', 'غزة', 'سبع', 'تلي', 'لوف', 'متج', 'دول', 'امر', 'قدم', 'خطب', 'فتة', 'لكن', 'قتل', 'خلل', 'قصف', 'اسرائيلي', 'قفل', 'غثي', 'قطع', 'غزة', 'وسف', 'لسط', 'وحد', 'قتل', 'ضمن', 'وظف', 'طبخ', 'علم', 'ركز', 'قطع', 'غزة', 'بين', 'اخر', 'حمل', 'جنس', 'جنب', 'تعدد', 'يقل', 'عصم', 'طه', 'ولد', 'سيف', 'ظهر', 'سمؤ', 'نسق', 'امن', 'خرج', 'غزة', 'كنا', 'ذهب', 'امر', 'وسف', 'ولد', 'خطب', 'فرح', 'بهذ', 'خطة', 'يمك', 'منع', 'قدر', 'هجم', 'وقع', 'سبع', 'اضي', 'اثر', 'حلة', 'وسع', 'غضب', 'وسط', 'دول', 'عذر', 'عنه', 'جيش', 'اسرائيلي', 'نسي', 'علم', 'سيف', 'يظل', 'وجد', 'قلب', 'يقل', 'شقق', 'عبد', 'رزق', 'طه', 'كلم', 'هتف', 'سي', 'يحك', 'اخه', 'كثر', 'فخر', 'وفي', 'وقت', 'اسى', 'فقد', 'قصص', 'قرح', 'نهي', 'سيف', 'وعد', 'خرج', 'غزة', 'سبع', 'تلي', 'لوف', 'اسم', 'سيف', 'طه', 'بلغ', 'عمر', 'عما', 'ولد', 'قطع', 'غزة', 'نقل', 'درس', 'عمل', 'امر', 'خرج', 'جمع', 'عجم', 'وعش', 'فتر', 'سيف', 'امر', 'قطع', 'عوم', 'فتح', 'عئل', 'شرع', 'طحن', 'حبب', 'يقل', 'شقق', 'شيء', 'ليك'

In [47]:
print(lemmatized_words1)

['سيف', 'ساعات', 'مقتله', 'سيف', 'موعد', 'للخروج', 'غزة', 'الأسبوع', 'التالي', 'لوفاته', 'متجها', 'دولة', 'الإمارات', 'ليتقدم', 'لخطبة', 'فتاة', 'لكنه', 'قتل', 'خلال', 'قصف', 'إسرائيلي', 'لقافلة', 'إغاثية', 'قطاع', 'غزة', 'وسيف', 'الفلسطيني', 'الوحيد', 'قتل', 'ضمن', 'موظفين', 'للمطبخ', 'العالمي', 'المركزي', 'قطاع', 'غزة', 'بينما', 'الآخرون', 'يحملون', 'جنسيات', 'أجنبية', 'متعددة', 'يقول', 'عصام', 'طه', 'والد', 'سيف', 'ظهرت', 'أسماؤنا', 'التنسيق', 'الأمني', 'للخروج', 'غزة', 'كنا', 'سنذهب', 'للإمارات', 'وسيف', 'ووالدته', 'لنخطب', 'فرحا', 'بهذه', 'الخطوة', 'يمكنه', 'منع', 'القدر', 'الهجوم', 'وقع', 'الأسبوع', 'الماضي', 'أثار', 'حالة', 'واسعة', 'الغضب', 'الأوساط', 'الدولية', 'واعتذر', 'عنه', 'الجيش', 'الإسرائيلي', 'نسي', 'العالم', 'سيف', 'سيظل', 'موجودا', 'قلوبنا', 'يقول', 'شقيقه', 'عبد', 'الرازق', 'طه', 'مكالمة', 'هاتفية', 'سي', 'يحكي', 'أخيه', 'بكثير', 'الفخر', 'وفي', 'الوقت', 'الأسى', 'لفقدانه', 'قصص', 'مقترحة', 'نهاية', 'سيف', 'موعد', 'للخروج', 'غزة', 'الأسبوع', 'التالي', 'لوفاته', 'اسم

### We can notice that the lemmatization is way more accurate which means that the words have meaning compared to stemming, even thought it takes a bit more time.

## 5- Apply Parts of Speech technics based on both Rule based and Machine learning approaches and NER.

In [58]:
pos_tagger = FarasaPOSTagger(interactive=True)
ner = FarasaNamedEntityRecognizer(interactive=True)

C:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|███████████████████████████████████████████████████████████████████████████████| 241M/241M [08:30<00:00, 472kiB/s]


[2024-04-07 23:36:44,925 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
[2024-04-07 23:37:11,919 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [65]:
# Convert the list of words into a single string
text1 = " ".join(filtered_words1)
text2 = " ".join(filtered_words1)
text3 = " ".join(filtered_words1)
# Perform part-of-speech tagging on the segmented text
pos_tags1 = pos_tagger.tag(text1)
pos_tags2 = pos_tagger.tag(text2)
pos_tags3 = pos_tagger.tag(text3)
# Perform named entity recognition on the segmented text
ner_tags1 = ner.recognize(text1)
ner_tags2 = ner.recognize(text2)
ner_tags3 = ner.recognize(text3)

# Print the segmented text, its part-of-speech tags, and named entity tags
for word, pos_tag1, ner_tag1 in zip(filtered_words1, pos_tags1, ner_tags1):
    print(f"Word: {word}, POS Tag: {pos_tag1}, NER Tag: {ner_tag1}")

Word: سيف, POS Tag: S, NER Tag: ?
Word: ساعات, POS Tag: /, NER Tag: ?
Word: مقتله, POS Tag: S, NER Tag: ?
Word: سيف, POS Tag:  , NER Tag: /
Word: موعد, POS Tag: ?, NER Tag: O
Word: للخروج, POS Tag: ?, NER Tag:  
Word: غزة, POS Tag: ?, NER Tag: ?
Word: الأسبوع, POS Tag: /, NER Tag: ?
Word: التالي, POS Tag: N, NER Tag: ?
Word: لوفاته, POS Tag: O, NER Tag: ?
Word: متجها, POS Tag: U, NER Tag: ?
Word: دولة, POS Tag: N, NER Tag: /
Word: الإمارات, POS Tag: -, NER Tag: O
Word: ليتقدم, POS Tag: M, NER Tag:  
Word: لخطبة, POS Tag: S, NER Tag: ?
Word: فتاة, POS Tag:  , NER Tag: ?
Word: لكنه, POS Tag: ?, NER Tag: ?
Word: قتل, POS Tag: ?, NER Tag: ?
Word: خلال, POS Tag: ?, NER Tag: ?
Word: قصف, POS Tag:  , NER Tag: /
Word: إسرائيلي, POS Tag: +, NER Tag: O
Word: لقافلة, POS Tag: ?, NER Tag:  
Word: إغاثية, POS Tag: ?, NER Tag: ?
Word: قطاع, POS Tag: /, NER Tag: ?
Word: غزة, POS Tag: N, NER Tag: ?
Word: وسيف, POS Tag: O, NER Tag: /
Word: الفلسطيني, POS Tag: U, NER Tag: O
Word: الوحيد, POS Tag: N, NER